In [1]:
import spacy
# python3 -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
# %pip install datasets
import numpy as np
from gensim.models import KeyedVectors
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
import datasets
nltk.download('stopwords')

/home/vyakhya/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vyakhya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
# Load pre-trained word embeddings (Word2Vec)
word2vec_model_path_pretrained = 'models/GoogleNews-vectors-negative300.bin.gz'
word2vec_model_pretrained = KeyedVectors.load_word2vec_format(word2vec_model_path_pretrained, binary=True)

In [8]:
from datasets import load_dataset
dataset = load_dataset("PiC/phrase_similarity")
train_data = dataset["train"]
validation_dataset = dataset["validation"]
test_data = dataset["test"]

Generating test split: 100%|██████████| 2000/2000 [00:00<00:00, 622485.01 examples/s]


In [9]:
stop_words = set(stopwords.words('english'))

In [1]:
from sklearn.metrics.pairwise import cosine_similarity

def get_phrase_embedding(phrase, word_embeddings):
    doc = nlp(phrase)
    embeddings = [word_embeddings[token.text] if token.text in word_embeddings else np.zeros(word_embeddings.vector_size) for token in doc]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word_embeddings.vector_size)

def phrase_similarity(phrase1, phrase2, word_embeddings):
    embedding1 = get_phrase_embedding(phrase1, word_embeddings)
    embedding2 = get_phrase_embedding(phrase2, word_embeddings)
    if np.any(embedding1) and np.any(embedding2):
        similarity = cosine_similarity([embedding1], [embedding2])[0][0]
        return similarity
    else:
        return None

def cosine_phrase_similarity(phrase1, phrase2, word_embeddings, threshold=0.5):
    similarity = phrase_similarity(phrase1, phrase2, word_embeddings)
    if similarity is not None:
        if similarity >= threshold:
            return 1
        else:
            return 0
    else:
        return "Unable to compute similarity"


In [16]:
# Example usage
phrase1 = "weird dude"
phrase2 = "strange woman"

classification = cosine_phrase_similarity(phrase1, phrase2, word2vec_model_pretrained)
print(f"Classification: {classification}")

Classification: 0


In [ ]:
predictions_cosine = []

# Iterate through the test dataset
for example in test_data:
    phrase1 = example["phrase1"]
    phrase2 = example["phrase2"]
    similarity_label = example["label"]
    prediction = cosine_phrase_similarity(phrase1, phrase2, word2vec_model_pretrained)
    predictions_cosine.append((phrase1, phrase2, similarity_label, prediction))


# Compute accuracy
correct_predictions = sum(1 for _, _, label, pred in predictions_cosine if label == pred)
total_predictions = len(predictions_cosine)
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy}")

In [18]:
# Generate phrase embeddings for the training dataset
X = []
y = []

import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

features = []
labels = []

i=0
train_len = len(train_data)

for example in train_data:
    if i>train_len//2:
      break
    i+=1
    phrase1_embedding = get_phrase_embedding(example['phrase1'], word2vec_model_pretrained)
    phrase2_embedding = get_phrase_embedding(example['phrase2'], word2vec_model_pretrained)

    if np.any(phrase1_embedding) and np.any(phrase2_embedding):
        combined_embedding = np.concatenate([phrase1_embedding, phrase2_embedding])
        features.append(combined_embedding)
        labels.append(example['label'])

X_train = np.array(features)
y_train = np.array(labels)

features = []
labels = []

i=0

for example in test_data:
    phrase1_embedding = get_phrase_embedding(example['phrase1'], word2vec_model_pretrained)
    phrase2_embedding = get_phrase_embedding(example['phrase2'], word2vec_model_pretrained)

    if np.any(phrase1_embedding) and np.any(phrase2_embedding):
        combined_embedding = np.concatenate([phrase1_embedding, phrase2_embedding])
        features.append(combined_embedding)
        labels.append(example['label'])

X_test = np.array(features)
y_test = np.array(labels)

# Train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.427


In [19]:
from sklearn.svm import SVC

# Train an SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Predict on the test set
y_pred_svm = svm_model.predict(X_test)

# Evaluate the SVM model
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("SVM Accuracy:", accuracy_svm)

SVM Accuracy: 0.4265


In [20]:
import gensim
import gensim.downloader as api
dataset = api.load("text8")
data = [i for i in dataset]

[==================================================] 100.0% 31.6/31.6MB downloaded


In [21]:
def tagged_document(list_of_list_of_words):
   for i, list_of_words in enumerate(list_of_list_of_words):
      yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])

training_data = list(tagged_document(data))


In [22]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=40, min_count=2, epochs=30)
model.build_vocab(training_data)
model.train(training_data, total_examples=model.corpus_count, epochs=model.epochs)
model.save("models/doc2vec")

print(model.infer_vector(['violent', 'means', 'to', 'destroy', 'the','organization']))

[ 0.04832448 -0.16907316 -0.32884434  0.07645246  0.07988791  0.0608741
 -0.01214247 -0.04433451 -0.30009368  0.06660686  0.13226591 -0.03787967
 -0.35997528 -0.06874289 -0.17011844  0.01362797  0.08046667  0.01126409
 -0.08782712 -0.03835297 -0.07367047 -0.01503135 -0.07407348  0.09473714
 -0.19559278 -0.15877326 -0.25511307 -0.04087844 -0.05472416 -0.174244
  0.09895404  0.11550625 -0.29199064 -0.3937337  -0.07769982  0.06879541
 -0.03918172 -0.20874064  0.03198326 -0.23841998]


In [23]:
model = gensim.models.doc2vec.Doc2Vec.load("models/doc2vec")

In [24]:
def get_vector_representation(phrase):
    preprocessed_phrase = gensim.utils.simple_preprocess(phrase)
    return model.infer_vector(preprocessed_phrase)

def get_phrase_cosine_similarty(phrase1, phrase2):
    vector1 = get_vector_representation(phrase1)
    vector2 = get_vector_representation(phrase2)
    return cosine_similarity([vector1], [vector2])[0][0]

In [35]:
correct_predictions = 0
total_predictions = 0
true_labels = []
predicted_labels = []
similarity_scores=[]
for data_entry in test_data:
    phrase1 = data_entry["phrase1"]
    phrase2 = data_entry["phrase2"]
    label = data_entry["label"]

    similarity_score = get_phrase_cosine_similarty(phrase1, phrase2)
    similarity_scores.append(similarity_score)
    threshold = 0.5

    # Classify as similar or not based on the threshold
    if similarity_score >= threshold:
        predicted_label = 1
    else:
        predicted_label = 0
    true_labels.append(label)
    predicted_labels.append(predicted_label)


    if predicted_label == label:
        correct_predictions += 1
    total_predictions+=1
accuracy = correct_predictions / total_predictions
print(f"Accuracy on the test set: {accuracy}")

Accuracy on the test set: 0.5075
